# # Sentiment Analysis Test (10 Samples)
# Testing the pipeline with a **High-Precision Persian Prompt**.

In [1]:
import sys
import os
import time
import pandas as pd
import ollama
from tqdm import tqdm

## Import modules from 'scripts'

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.preprocessor import TextPreprocessor

## Configuration

In [3]:
DATA_DIR = "../data"
OUTPUT_DIR = "../data/processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)
MODEL_NAME = "llama3"
TARGET_FILE = "kafiha_messages.csv" # Example file

## Data Loading & Cleaning

In [ ]:
def load_and_clean_data(filename, sample_size=None):
    """
    Loads CSV, cleans text FIRST, filters short texts, and THEN samples.
    This ensures we get exactly 'sample_size' valid rows.
    """
    path = os.path.join(DATA_DIR, filename)
    df = pd.read_csv(path)
    
    # Fill NaNs
    df['text'] = df['text'].fillna('')
    df['emoji_reactions'] = df['emoji_reactions'].fillna('')
    
    # Initialize Preprocessor
    preprocessor = TextPreprocessor()
    
    print(f"Preprocessing all data in {filename} to find valid samples...")
    # Clean all rows first
    tqdm.pandas(desc="Cleaning Text")
    df['clean_post_text'] = df['text'].progress_apply(preprocessor.clean_text)
    df['clean_reactions'] = df['emoji_reactions'].apply(preprocessor.clean_reactions)
    
    # Filter: Remove short texts (noise) BEFORE sampling
    # We increase the threshold to 15 to ensure meaningful sentences
    df_clean = df[df['clean_post_text'].str.len() > 15].copy()
    
    # Now take the sample from the clean pool
    if sample_size:
        if len(df_clean) >= sample_size:
            df_clean = df_clean.head(sample_size).copy()
            print(f"Successfully selected {sample_size} valid samples.")
        else:
            print(f"Warning: Only found {len(df_clean)} valid samples.")
    
    return df_clean

## Sentiment Inference with Precise Persian Prompt

In [ ]:
def get_sentiment(text, reactions):
    """
    Uses Few-Shot Prompting to achieve maximum accuracy.
    We provide examples to the model so it understands nuances like 'Heartbreak' vs 'Happiness'.
    """
    # Context
    prompt_text = f"متن پست: {text}\nواکنش‌ها: {reactions}"
    
    # --- FEW-SHOT PROMPT (Maximum Accuracy) ---
    system_prompt = (
        "تو یک تحلیلگر دقیق احساسات فارسی هستی. بر اساس مثال‌های زیر، پست جدید را طبقه‌بندی کن.\n"
        "دسته‌بندی‌ها: ['خوشحال', 'ناراحت', 'عصبانی', 'مضطرب', 'خنثی', 'نگران']\n\n"
        "--- مثال‌های آموزشی ---\n"
        "مثال ۱ (تبلیغات/خبر):\n"
        "متن: تور استانبول ویژه بلک فرایدی با قیمت مناسب هتل ۵ ستاره.\n"
        "برچسب: خنثی\n\n"
        "مثال ۲ (شکست عشقی/غم):\n"
        "متن: فراموش کردنت سخته عزیزم، هنوز دوستت دارم ولی نیستی که ببینی.\n"
        "برچسب: ناراحت\n\n"
        "مثال ۳ (طنز/خوشحالی):\n"
        "متن: بالاخره تموم شد! وای چقدر خوشحالم که این پروژه رو تحویل دادم.\n"
        "برچسب: خوشحال\n\n"
        "مثال ۴ (عصبانیت/اعتراض):\n"
        "متن: چرا اینقدر وضعیت اینترنت افتضاحه؟ شورشو درآوردن دیگه واقعا خسته شدیم.\n"
        "برچسب: عصبانی\n\n"
        "مثال ۵ (نگرانی/اضطراب):\n"
        "متن: وای آخر ماه شد و هنوز حقوق نریختن، قسط‌ها رو چیکار کنم؟ استرس دارم.\n"
        "برچسب: نگران\n\n"
        "--- پایان مثال‌ها ---\n"
        "حالا فقط برچسب پست زیر را بنویس (بدون هیچ توضیح اضافه):"
    )

    try:
        response = ollama.chat(model=MODEL_NAME, messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': prompt_text},
        ])
        
        label = response['message']['content'].strip()
        
        # Validation
        valid_labels = ['خوشحال', 'ناراحت', 'عصبانی', 'مضطرب', 'خنثی', 'نگران']
        for v in valid_labels:
            if v in label:
                return v
        # If the model adds punctuation, clean it
        for v in valid_labels:
            if v in label: return v
            
        return "خنثی" # Fallback
        
    except Exception as e:
        print(f"Error: {e}")
        return "Error"

## 1. Load new data (Exactly 10 samples)
### Make sure to run this line again to get the new filtered dataframe

In [ ]:
df_test = load_and_clean_data(TARGET_FILE, sample_size=10)

Running Analysis on 10 samples...


NameError: name 'df_test' is not defined

## 2. Run Analysis

In [ ]:
print("\nRunning Analysis on 10 samples...")
sentiments = []
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    label = get_sentiment(row['clean_post_text'], row['clean_reactions'])
    sentiments.append(label)

df_test['sentiment'] = sentiments

## 3. Show results

In [ ]:
print("\n--- Final Results ---")
print(df_test[['clean_post_text', 'sentiment']])

# 4. Save

In [ ]:
df_test.to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False, encoding='utf-8-sig')